# Timelag Differential Equation Tutorial

The purpose of this notebook is to demonstrate the use of the physics-based fuel moisture model used with `wrfxpy`. This model is used as a benchmark for the machine learning methods. The model is a time-lag ODE plus a Kalman filter for assimilating fuel moisture observations.

## Model Background

The physics-based model used within WRF-SFIRE is a timelag ODE. Data assimilation is done through Augmented Kalman filter. Model state is extended to include equilibrium bias correction term.

* **Inputs**: wetting/drying equilibrium moisture content and hourly rainfall, and optional FMC data for data assimilation
* **Spinup**: model is run with data assimilation for a number of spinup hours for equilibrium  bias to stabilize, this is analogous to training an ML model
* **Forecast**: model is run with no data assimilation after set number of spinup hours
* **How Model is Applied**: ODE+KF applied pointwise, or independently at some set of grid nodes. In this project, the ODE+KF will be run at the location of RAWS sites, using the observed RAWS data for spinup data assimilation. NOTE: this is "best case" scenario for the model, since in production spatially interpoalted FMC used for spinup data assimilation

For more info, see ___

## ODE+KF in this Project

**Workflow:**
- Retrieve fmda data: gets data from API or stash, interpolates missing observations to regular hourly intervals
- Build fmda ML data: merges data sources and applies filters
- Define a cross validation test period and test locations (RAWS STIDS)
- Based on CV above, get needed data from built ML data

**ODE Modeling:**
* Run on 72 hour stretches (24 spinup, 48 val)
* Get test station list used by other models
* For those test stations, use `get_sts_and_times` accounting for the spinup period
    * So adjust test times by subtracting 24 hours to account for spinup

## Setup

In [ ]:
import os.path as osp
import json
import sys
import numpy as np
import pandas as pd
sys.path.append('../src')
from utils import Dict, read_yml, read_pkl, str2time, print_dict_summary, time_range, rename_dict
import data_funcs
import models.moisture_models as mm
import matplotlib.pyplot as plt

## Create Data

In [ ]:
ml_data = read_pkl("../data/test_data/test_ml_dat.pkl")

In [ ]:
# Get Test Cross-Val Period
train_times, val_times, test_times = data_funcs.cv_time_setup("2023-01-05T00:00:00Z", 
                                                train_hours=48*2, forecast_hours=48)

In [ ]:
# Get Test Station List
stids = [*ml_data.keys()]

tr_sts, val_sts, te_sts = data_funcs.cv_space_setup(stids, random_state=42)

In [ ]:
ode_data = data_funcs.get_ode_data(ml_data, te_sts, test_times)

In [ ]:
print(ode_data.keys())

In [ ]:
print(ode_data["YLSU1"].keys())

In [ ]:
ode_data["YLSU1"]["data"]

## Run Model

Model object creator defined in `models/moisture_models`. Has hyperparameters associated with model, such as fixed covariance matrices

In [ ]:
ode = mm.ODE_FMC()

In [ ]:
ode.params

### Run Single Case

In [ ]:
u = ode.run_model_single(ode_data["YLSU1"], hours=72, h2=24)

In [ ]:
print(u.shape)

In [ ]:
plt.plot(u[0,:])

In [ ]:
# Print RMSE for Period
ode.eval(u[0,:], ode_data["YLSU1"]["data"].fm.to_numpy())

## Run Whole Dictionary

In [ ]:
m, errs = ode.run_model(ode_data, hours=72, h2=24)

In [ ]:
# Should be shape (n_locations, forecast_hours, 1)
print(m.shape)

In [ ]:
print(errs)